In [1]:
import pygame
import cv2
import math
import random
import sys
from collections import Counter
from pygame.locals import *
pygame.init()

pygame 2.0.0.dev6 (SDL 2.0.10, python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


(6, 0)

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Input, Flatten, Dense, Lambda, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.models import Model
from easydict import EasyDict as edict

In [6]:
config = edict()

config.CHECKPOINT_DIR = '/Users/vijay/Downloads/ew/snake_game_ckpts/'
config.NUM_EPOCHS     = 5000
config.BATCH_SIZE     = 64
config.EPSILON        = 0.9
config.BUFFER_SAMPLE_RATE = 0.8
config.BUFFER_SAMPLE_RATE_DECAY_RATE = 0.1
config.DISCOUNT_FACTOR = 0.99

In [3]:
import sys
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install easydict

  Using cached easydict-1.9.tar.gz (6.4 kB)
  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6349 sha256=0d170b5408d5498114f0cf5ae435b42d96d9040b0f85585736a168f03f44adf8
  Stored in directory: /Users/vijay/Library/Caches/pip/wheels/d3/e0/e9/305e348717e399665119bd012510d51ff4f22d709ff60c3096
Successfully built easydict


In [20]:
# import sys
# !{sys.executable} -m pip install opencv-python
# !{sys.executable} -m pip install pygame
!python3 -m pip install pygame -U --user

Requirement already up-to-date: pygame in /Users/vijay/.local/lib/python3.8/site-packages (2.0.0.dev6)


pygame 2.0.0.dev6 (SDL 2.0.10, python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
class settings:
    def __init__(self):
        
        self.screen_color = (100, 100, 100)
        self.screen_width = 200
        self.screen_height = 200
        self.run_game = True
        self.snake_nodes_group = []
        
        
class Snake_Node:
    def __init__(self, node_num, x, y, color, radius):
        
        self.snake_node_num = node_num
        self.x = x
        self.y = y
        self.color = color
        self.radius = radius
        self.direction = 'up'
        
    def draw_snake_node(self, screen):
#         snake_node = pygame.draw.circle(screen, self.color, (self.x, self.y), self.radius)
        snake_node = pygame.draw.rect(screen, self.color, (self.x, self.y, 10, 10), 1)
        return snake_node
    
    
    
class Snake_Head:
    def __init__(self, x, y, color, radius):
        self.x = x
        self.y = y
        self.color = color
        self.radius = radius
#         self.moving_direction = 'up'
    
    def draw_snake_head(self, screen):
        snake_head = pygame.draw.rect(screen, self.color, (self.x, self.y, 10, 10))
#         snake_head = pygame.draw.circle(screen, self.color, (self.x, self.y), self.radius)
        return snake_head
        
        
        
        

In [8]:


class Game_functions:
    
    ####################################
    
    def __init__(self):
        self.num_of_steps_in_curr_episode = 0
        self.start_training_gap_period = False
        self.is_apple_there = False
        self.game_settings = settings()
        self.divide_screen_into_segments()
        self.num_of_snake_nodes = 0
        self.snake_head_moving_direction = ' '
        self.xy_list = []
        self.radius = 10
        self.rect_width = 5
        self.rect_height = 5
        self.full_reward = 0
#         self.full_reward = False
        self.img_count = 0
        self.prev_distance = -1
        self.new_game = True
        self.score    = 0
        self.img_dir  = '/Users/vijay/Downloads/ew/'
        
        self.set_up_the_screen()
        
    ####################################
    
    def divide_screen_into_segments(self):
        self.segment_indices = []
        for i in range(0, 200):
            if i % 10 == 0:
                self.segment_indices.append(i)
                
                
    
    ####################################
    
    def draw_snake_nodes_xy(self, from_list):
        
        if from_list:
            for index, xy_tuple in enumerate(self.xy_list):
                if index != 0:
                    self.create_snake_head_or_apple(xy_tuple[0], xy_tuple[1], 5, False)
                    pygame.display.update()
        
        
    
    
    ###################################
    
    def get_new_xy_for_apple(self):
        x, y = random.choice(self.segment_indices),random.choice(self.segment_indices)
        if (x, y) not in self.xy_list:
            return x, y
        else:
            return self.get_new_xy_for_apple()
        
        
    ####################################
    
    def create_snake_head_or_apple(self, x, y, is_apple):
        
        if not is_apple:
            color = (255,255,255)
            
            if self.num_of_snake_nodes != 0:
                
                snake_node_obj = Snake_Node(self.num_of_snake_nodes, x, y, color, self.radius)
                snake_node = snake_node_obj.draw_snake_node(self.screen)
                
                
            elif self.num_of_snake_nodes == 0:
                snake_head_obj = Snake_Head(x, y, (0, 255, 0), self.radius)
                self.snake_head = snake_head_obj.draw_snake_head(self.screen)

                
            self.num_of_snake_nodes = self.num_of_snake_nodes + 1
            
            if self.num_of_snake_nodes < 5:
                pygame.time.wait(80)
            else:
                pygame.time.wait(10)
            
        if is_apple:
            color = (255, 0, 0)
            self.apple = pygame.draw.rect(self.screen, color, (x, y, 10, 10))
#             self.apple = pygame.draw.circle(self.screen, color, (x, y), self.radius)
            self.is_apple_there = True
            self.apple_x = x
            self.apple_y = y


    
    
    
        
    ####################################
    
    def start_a_new_game(self):
#         print('start a new game')
        if self.new_game:
            self.is_apple_there     = False
            self.num_of_snake_nodes = 0
            self.xy_list.clear()
            self.screen.fill((100, 100, 100))
            self.prev_distance      = -1
#             self.img_count          = 0
            self.full_reward        = 0
            self.score              = 0
            
            x, y = random.choice(self.segment_indices),random.choice(self.segment_indices)
            self.xy_list.append((x, y))
            self.draw_rect_at_new_location()
            self.new_game           = False
#             self.set_up_the_screen()

    ####################################
    
    def check_if_snake_is_at_boundaries(self, new_x, new_y, direction):

        if direction == 'up':
            if new_y  < 0:
                print('wall')
                return True
    

        elif direction == 'down':
            if new_y  > self.game_settings.screen_width:
                print('wall')
                return True
    

        elif direction == 'left':
            if new_x  < 0:
                print('wall')
                return True
  

        elif direction == 'right':
            if new_x  > self.game_settings.screen_width:
                print('wall')
                return True
        
        
        return False






    ####################################

    def draw_rect_at_new_location(self):
        
        self.screen.fill((100, 100, 100))
        self.num_of_snake_nodes = 0
        
        for xy_tuple in self.xy_list:        
            self.create_snake_head_or_apple(xy_tuple[0], xy_tuple[1], False)
    
        
        self.create_apple()
        pygame.display.update()


    
    ####################################
    
    def get_new_x_y_for_moving(self, x, y, direction):
        
        if direction == 'up':
            y = y - self.radius
            

        elif direction == 'down':
            y = y + self.radius
            

        elif direction == 'left':
            x = x - self.radius
            

        elif direction == 'right':
            x = x + self.radius
            
        return x, y
    
    
#     ###################################
    
    def move_nodes(self, head_direction):
        prev_tuple = (-1, -1)
        curr_tuple = (-1, -1)
        for index, xy_tuple in enumerate(self.xy_list):
            if index == 0:
                prev_tuple = xy_tuple
            elif index != 0:
                curr_tuple = self.xy_list[index]
                self.xy_list[index] = prev_tuple
                prev_tuple = curr_tuple
        
    
    
    ####################################
    
    def check_if_snake_ran_into_its_own_body(self):
        
        tuple_counter = Counter(self.xy_list)
        if tuple_counter[self.xy_list[0]] != 1:
            return True
        
        return False
        
 
    
    ####################################
    
    def move_snake_head_new(self, direction):
        
        '''
        First, get the new xy for the snake head to move
        check whether the snake head is at the boundaries wrt new xy
        if not, get new xy for all the snake nodes 
        update the xy_list with all the new xy values of both nodes and head
        after updating, check, after moving, if the head bumped into its own body
        '''
        x, y = self.xy_list[0][0], self.xy_list[0][1]
        new_x, new_y = self.get_new_x_y_for_moving(x, y, direction)
        is_snake_at_boundary = self.check_if_snake_is_at_boundaries(new_x, new_y, direction)

        if not is_snake_at_boundary:
            
            self.move_nodes(direction)
            self.xy_list[0] = (new_x, new_y)
            snake_ran_into_its_own_body = self.check_if_snake_ran_into_its_own_body()
            if not snake_ran_into_its_own_body:
                self.new_game = False
                self.draw_rect_at_new_location()
            else:
                '''
                if the snake bumped into its own body, then a reward of -1 has to be given to the snake
                '''
                self.full_reward = -1
                self.new_game    = True
                self.end_game()
#                 self.start_a_new_game()

        else:
            '''
            if the snake bumped into any boundary, then a reward of -1 has to be given to the snake
            '''
            self.full_reward = -1
            self.new_game    = True
            self.end_game()
#             self.start_a_new_game()

        pygame.display.update()

        
    

    ####################################
    
    def create_apple(self):
        
        if not self.is_apple_there:
            x, y = self.get_new_xy_for_apple()
#             x, y = random.choice(self.segment_indices),random.choice(self.segment_indices)
            
            self.create_snake_head_or_apple(x, y, True)
        
        if self.is_apple_there:
            self.create_snake_head_or_apple(self.apple_x, self.apple_y, True)
            
        pygame.display.update()
    
    
    
    
    #####################################
    '''
    if an action is supplied (direction) and in that direction, if there is a node in that direction, then stop moving it
    
    '''
    def check_whether_to_move_nodes_or_not(self, direction):
        if direction == 'up':
            if self.xy_list[0][0] == self.xy_list[1][0] and self.xy_list[0][1] == self.xy_list[1][1] + self.radius:
                return False
        
        elif direction == 'down':
            if self.xy_list[0][0] == self.xy_list[1][0] and self.xy_list[0][1] == self.xy_list[1][1] - self.radius:
                return False
        
        elif direction == 'left':
            if self.xy_list[0][0] == self.xy_list[1][0] + self.radius and self.xy_list[0][1] == self.xy_list[1][1]:
                return False
            
        elif direction == 'right':
            if self.xy_list[0][0] == self.xy_list[1][0] - self.radius and self.xy_list[0][1] == self.xy_list[1][1]:
                return False
        
        return True
    
    
    
    #####################################
    
    def get_new_xy_for_new_node_after_eating_apple(self):
        
        last_node_x, last_node_y = self.xy_list[-1][0], self.xy_list[-1][1]
        new_x, new_y = -1, -1
        
        if last_node_x + self.radius < self.game_settings.screen_width:
            if (last_node_x + self.radius, last_node_y) not in self.xy_list: 
                new_x = last_node_x + self.radius
                new_y = last_node_y
        
        elif last_node_x - self.radius > 0:
            if (last_node_x - self.radius, last_node_y) not in self.xy_list:
                new_x = last_node_x - self.radius
                new_y = last_node_y
        
        
        elif last_node_y - self.radius > 0 :
            if (last_node_x, last_node_y - self.radius) not in self.xy_list: 
                new_x = last_node_x
                new_y = last_node_y -self.radius5
            
        elif last_node_y + self.radius < self.game_settings.screen_width:
            if (last_node_x, last_node_y + self.radius) not in self.xy_list:
                new_x = last_node_x
                new_y = last_node_y + self.radius
            
        
        return new_x, new_y
    
    
    
    #####################################
    
    def add_just_eaten_apple_to_the_snake_end(self):
        
        new_node_x, new_node_y = self.get_new_xy_for_new_node_after_eating_apple()
        self.xy_list.append((new_node_x, new_node_y))
        self.is_apple_there = False
        self.draw_rect_at_new_location()
        self.start_training_gap_period = True
        
        
    
    
    
    
    
    
    
    
    #####################################
    
    '''
    if apple's xy and snake_head's xy (first value in xy_list), then we consider that snake has eaten apple
    '''
    def is_apple_eaten(self):
        
        if self.is_apple_there:
            if self.apple_x == self.xy_list[0][0] and self.apple_y == self.xy_list[0][1]:
                '''
                if apple is eaten, a full reward of 1 has to be given to the snake
                '''
                self.full_reward = 1
                self.score = self.score + 10
                self.add_just_eaten_apple_to_the_snake_end()
                
    
    
    
    ###################################
    
    def check_if_snake_moved_closer_to_apple(self):
        
        snake_head_xy = self.xy_list[0]
        apple_xy      = (self.apple_x, self.apple_y)
        
        curr_distance = math.sqrt( ((snake_head_xy[0] - apple_xy[0]) ** 2) + ((snake_head_xy[1] - apple_xy[1]) ** 2))
        
        if self.prev_distance == -1:
            self.prev_distance = curr_distance
            return True
        
        else:
            if curr_distance < self.prev_distance:
                return True
            else: 
                False
        

    
    ###################################
    
    def get_reward(self):
        '''
        -- if snake_head moved towards apple, reward = 0.2
        --                            if not, reward = -0.2
        --- if snake bumped into its own body or bumped into boundaries, reward = -1
        --- if snake eats apple, reward = 1
        --- if snake cannot move, then reward = -0.1
        '''
        if self.full_reward == 0:
            moved_closer = self.check_if_snake_moved_closer_to_apple()
            if moved_closer:
                return 0.2
            
            elif not moved_closer:
                return -0.2
        
        else:
            return self.full_reward
        
#         elif self.full_reward == -0.1:
#             return -0.1
            
#         elif self.full_reward == -1:
#             return -1
        
#         elif self.full_reward == 1:
#             return 1
        
        
    
    
    ##################################
    
    def get_curr_state_of_the_game(self):
        
        '''
        -- Stacking last 4 screenshots of the image as one
        -- if the num of screenshots is less than 4, then the last screenshot has to be appended 
            required num of times
        --- if the num of screenshots is greater than 4, then the recent last 4 screenshots has to be 
            appended
        '''
        img_ids = []
        if self.img_count <= 4:
            for id in range(self.img_count - 1):
                img_ids.append(id)
            remaining = 4 - (self.img_count - 1)
            for id in range(remaining):
                img_ids.append(img_ids[-1])

        else:
            for id in range(4):
                img_ids.append(self.img_count - (id + 1))
        
        img_ids.reverse()
        
        for id in img_ids:
            self.curr_screenshots_paths.append(self.img_dir + str(id) + '.jpg')
        reward = self.get_reward()        
        end_game = self.new_game
        
#         self.curr_screenshot_path = curr_screenshot_path
        self.curr_reward = reward
        self.end_game    = end_game
        
    

        
    ###################################    
        
    def set_up_the_screen(self):
        pygame.display.init()
        self.screen = pygame.display.set_mode((self.game_settings.screen_width, self.game_settings.screen_height))
        pygame.display.flip()
        self.start_a_new_game()
        pygame.image.save(self.screen, self.img_dir + str(self.img_count) + '.jpg')
        self.img_count = self.img_count + 1
#         self.get_curr_state_of_the_game()
        img_ids = []
        curr_screenshot_path = self.img_dir + str(self.img_count - 1) + '.jpg'
        for id in range(4):
            img_ids.append(curr_screenshot_path)
        
        self.curr_screenshots_paths = img_ids
        self.curr_reward = 0
        self.end_game = False
#         image = pygame.image.tostring(self.screen, 'RGB')
#         cv2.imshow('img', image)
        
    
    
    def end_the_game(self):
        pygame.display.quit()
        pygame.quit()
        exit()
        
    ####################################
    
    def run(self, action):
#         while self.game_settings.run_game:
        try:
            
            apple_eaten = False
            for event in pygame.event.get():

                if event.type == QUIT:
                    self.game_settings.run_game = False
                    pygame.display.quit()
                    pygame.quit()
                    exit()
            
            if self.num_of_snake_nodes == 1:
                move_nodes = True
            
            keys = pygame.key.get_pressed()
            

            ############
#                 if keys[pygame.K_UP]:

#                     action = 'up'
#                     print(action)
            if action == 'up':
                if self.num_of_snake_nodes > 1:
                    move_nodes = self.check_whether_to_move_nodes_or_not(action)
                    if not move_nodes:
                        self.full_reward = -0.1


                if move_nodes:
                    
                    self.move_snake_head_new(action)
                    pygame.image.save(self.screen, self.img_dir + str(self.img_count) + '.jpg')
                    self.img_count = self.img_count + 1
                    self.is_apple_eaten()


            ##############
#                 elif keys[pygame.K_DOWN]:
#                     action = 'down'
#                     print(action)
            elif action == 'down':
                if self.num_of_snake_nodes > 1:
                    move_nodes = self.check_whether_to_move_nodes_or_not(action)
                    if not move_nodes:
                        self.full_reward = -0.1

                if move_nodes:
                    
                    self.move_snake_head_new(action)
                    pygame.image.save(self.screen, self.img_dir + str(self.img_count) + '.jpg')
                    self.img_count = self.img_count + 1
                    self.is_apple_eaten()


            ##############
#                 elif keys[pygame.K_LEFT]:
#                     action = 'left'
#                     print(action)
            elif action == 'left':
                if self.num_of_snake_nodes > 1:
                    move_nodes = self.check_whether_to_move_nodes_or_not(action)
                    if not move_nodes:
                        self.full_reward = -0.1

                if move_nodes:
                    
                    self.move_snake_head_new(action)
                    pygame.image.save(self.screen, self.img_dir + str(self.img_count) + '.jpg')
                    self.img_count = self.img_count + 1
                    self.is_apple_eaten()



            ###########
#                 elif keys[pygame.K_RIGHT]:
#                     action = 'right'
#                     print(action)
            elif action == 'right':
                if self.num_of_snake_nodes > 1:
                    move_nodes = self.check_whether_to_move_nodes_or_not(action)
                    if not move_nodes:
                        self.full_reward = -0.1

                if move_nodes:

                    self.move_snake_head_new(action)
                    pygame.image.save(self.screen, self.img_dir + str(self.img_count) + '.jpg')
                    self.img_count = self.img_count + 1
                    self.is_apple_eaten()


            pygame.display.flip()
            self.get_curr_state_of_the_game()

        except:
            self.end_the_game()


    
    
    
# game_functions = Game_functions()
# # game_functions.run('test')
# for i in range(500):
#     actions = ['up', 'left', 'right', 'down']
#     action_index = random.randint(0, 3)
#     action = actions[action_index]
#     game_functions.run(action)
    

wall


In [15]:

class Q_values_approximator_model:
    def __init__(self):
        pass
        
    def normalize_to_range_01(self, img):
        return tf.cast(img, tf.float32) / 255.0
    
    def get_q_values(self):
        input_data = Input(shape = (64, 64, 12))
        data = Lambda(self.normalize_to_range_01)(input_data)
        data = Conv2D(filters = 32, kernel_size = 7, strides = (4, 4), padding = 'SAME')(data)
        data = ReLU()(data)
        
        data = Conv2D(filters = 64, kernel_size = 5, strides = (2, 2), padding = 'SAME')(data)
        data = ReLU()(data)
        
        data = Conv2D(filters = 128, kernel_size = 3, strides = (2, 2), padding = 'SAME')(data)
        data = ReLU()(data)
        
        data = Flatten()(data)
        data = Dense(512)(data)
        data = ReLU()(data)
        
        data = Dense(4)(data)
        
        return tf.keras.Model(input_data, data)
        

    

In [ ]:
import numpy as np
q_values_predictor = Q_values_approximator_model()
in_data = np.arange(64 * 64*12).reshape(64, 64, 12)
out_data = q_values_predictor.get_q_values()(np.expand_dims(in_data, axis = 0))
print(out_data.numpy())


In [ ]:
class training:
    def __init__(self):
        
        
        self.game_controls = Game_functions()
        self.training_gap_steps = 0
        self.experiences_buffer_1   = []
        self.experiences_buffer_2 = []
        self.observation_period_threshold = 5
        self.num_of_steps_completed_in_obv_period = 0
        self.actions = ['up', 'down', 'right', 'left']
        
        q_values_predictor = Q_values_approximator_model()
        self.q_values_pred_model = q_values_predictor.get_q_values()
        self.model_optimizer = Adam(learning_rate = 0.001)
        self.checkpoint = tf.train.Checkpoint(curr_epoch = tf.Variable(0),
                                              model = self.q_values_pred_model, 
                                              optimizer = self.model_optimizer
                                             )
        self.checkpoint_manager = tf.train.CheckpointManager(self.checkpoint, 
                                                            directory = config.CHECKPOINT_DIR,
                                                            max_to_keep = 3)
        
        
        
        
        
    def preprocess_and_stack_images(self, curr_game_images_paths):
        images = []
        for image_path in curr_game_images_paths:
            image = cv2.resize(cv2.imread(image_path), (64, 64))
            images.append(image)
        
        stacked_images = np.concatenate((images[0], images[1], images[2], images[3]), axis = 2)
        return stacked_images
    
    
    
    
    '''
    After deciding upon action to be taken, run the game for one step
    Then, collect the next state images paths, reward received and whether the next state is terminal
    '''
    def perform_a_step_in_an_episode(self, action):
        
        self.game_controls.run(action)
        game_imgs_paths_after_action = self.game_controls.curr_screenshots_paths
        reward_after_action = self.game_controls.curr_reward
        end_game_after_action = self.game_controls.end_game
        start_training_gap_period = self.start_training_gap_period
        return game_imgs_paths_after_action, reward_after_action, end_game_after_action, start_training_gap_period
    
    
    
    
    def determing_num_of_training_gap_steps_after_snake_eating_apple(self):
        length_of_snake = self.game_controls.score 
        k = 10
        p = 0.4
        q = 0.2
        if length_of_snake <= k:
            self.training_gap_steps = 6
        elif length_of_snake > k:
            self.training_gap_steps = math.ceil((p * length_of_snake) + q)
    
    
    
    
    
    '''
    --- Get an action and run the game as per the action
    --- Now, record the updated state of the game by getting the curr screenshot of the screen, 
         current reward and also whether the current state resulted in quitting game which happens
         when snake bumps into boundaries or ran into itself.

    '''
    def add_experiences_to_buffers_after_running_game_for_one_step(self, action, curr_state_imgs_paths):
        
        next_state_imgs_paths, reward_after_action, end_game, start_training_gap_period = self.perform_a_step_in_an_episode(action)
        
        if reward_after_action > 0:
            self.experiences_buffer_1.append(curr_state_imgs_paths, action, reward_after_action, next_state_imgs_paths, end_game)

        else:
            self.experiences_buffer_2.append(curr_state_imgs_paths, action, reward_after_action, next_state_imgs_paths, end_game)

        
            
        
        return next_state_imgs_paths, end_game, start_training_gap_period
    
    
    
    
    
    def start_exploration_in_observation_period(self):
        
        num_of_training_gap_steps_completed = 0
        enter_training_gap_period = False
        curr_state_imgs_paths = self.game_controls.curr_screenshots_paths
        curr_reward           = self.game_controls.curr_reward
        end_game              = self.game_controls.end_game
        start_training_gap    = self.game_controls.start_training_gap_period
#         curr_state           = self.preprocess_and_stack_images(curr_game_imgs_paths)
        
        
        while self.num_of_steps_completed_in_obv_period < self.observation_period_threshold and not end_game:
            
            '''
            -- so, when the snake eats an apple, then we should immediately enter into a training_gap_period during which no experiences are
                stored into experience buffers.
            
            -- if start_training_gap and enter_training_gap_period == False, then we haven't entered the training_gap and therefore set
                enter_training_gap_period = True
            
            -- if not start_training_gap, then add experiences to the buffer. if in training_gap_period, then just perform single step in the game
                without storing experiences in the experience buffers. Also, increase the number of steps completed in trainig_gap_period
            
            -- if the num_of_steps in training_gap is equal to the pre-determined num_of_steps in training_gap_period, then exit training_gap_period
                by setting enter_training_gap_period = False and self.game_controls.start_training_gap_period = False
            
            '''
            
            if start_training_gap and not enter_training_gap_period:
                enter_training_gap_period = True
                self.determing_num_of_training_gap_steps_after_snake_eating_apple()
            
            if enter_training_gap_period and (num_of_training_gap_steps_completed == self.training_gap_steps - 1):
                enter_training_gap_period = False
                self.game_controls.start_training_gap_period = False
            
            action = self.actions[random.randint(0, 3)]
#             next_state, end_game = self.add_experiences_to_buffers_after_running_game_for_one_step(action, curr_state)

            if not enter_training_gap_period:
        
                next_state_imgs_paths, end_game, start_tranining_gap = self.add_experiences_to_buffers_after_running_game_for_one_step(action, curr_state_imgs_paths)
                curr_state_imgs_paths = next_state_imgs_paths
            
            elif enter_training_gap_period:
                
                next_state_imgs_paths, reward_after_action, end_game, start_training_gap = self.perform_a_step_in_an_episode(action)
                num_of_training_gap_steps_completed = num_of_training_gap_steps_completed + 1
                curr_state_imgs_paths = next_state_imgs_paths
                
            self.num_of_steps_completed_in_obv_period = self.num_of_steps_completed_in_obv_period + 1
            
                
        
        
    def restore_checkpoint(self):

        if self.checkpoint_manager.latest_checkpoint:
            self.checkpoint.restore(self.checkpoint_manager.latest_checkpoint)
            print('restored checkpoint successfully at epoch ' + str(self.checkpoint.curr_epoch.numpy()))
        else:
            print('No checkpoint restoration')



    def epsilon_greedy_policy_for_action(self, curr_state_imgs_paths):
        if random.random() < config.EPSILON:
            return self.actions[random.randint(0, 3)]
        else:
            curr_state_imgs = self.preprocess_and_stack_images(curr_state_imgs_paths)
            q_values = self.checkpoint.model(curr_state_imgs, training = False)
            return self.actions[np.argmax(q_values)]



    def adjust_buffer_sample_rate(self, epoch):
        if epoch % 1000 == 0:
            decay_value = config.BUFFER_SAMPLE_RATE * config.BUFFER_SAMPLE_RATE_DECAY_RATE
            new_buffer_sample_rate = config.BUFFER_SAMPLE_RATE - decay_value
            if not new_buffer_sample_rate > 0.5:
                config.BUFFER_SAMPLE_RATE = new_buffer_sample_rate





    def sample_mini_batches_from_experience_buffers(self):

        num_samples_from_buffer_1 = int(64 * config.BUFFER_SAMPLE_RATE)
        num_samples_from_buffer_2 = 64 - num_samples_from_buffer_1

        random_samples_from_buffer_1 = random.sample(self.experiences_buffer_1, num_samples_from_buffer_1)
        random_samples_from_buffer_2 = random.sample(self.experiences_buffer_2, num_samples_from_buffer_2)

        return random_samples_from_buffer_1, random_samples_from_buffer_2






    def get_target_labels_for_samples(self, buffer_samples):
        '''
        experience[0] = curr_State_imgs_paths
        experience[1] = action [This is a text 'up', 'down', 'left', 'right']
        experience[2] = reward_after_action
        experience[3] = next_state_imgs_paths
        experience[4] = end_game
        '''
        label_values = []
        for experience in buffer_samples:
            if experience[4]: # is the episode reached terminal state - end_game
                label_values.append(experience[2])

            else:
                next_state_imgs = self.preprocess_and_stack_images(experience[3])
                q_values_of_actions = self.checkpoint.model(next_state_imgs)
                print('predicted values in train_step ' + str(q_values_of_actions.numpy()))
                target_value = experience[2] + (config.DISCOUNT_FACTOR * np.max(q_values_of_actions[0]))
                label_values.append(target_value)

        return label_values




    def get_actual_labels_for_samples(self, buffer_samples):
        actual_labels = []
        for experience in buffer_samples:
            curr_state_imgs_paths = experience[0]
            action = experience[1]
            action_index = self.actions.index(action)

            curr_state_imgs = self.preprocess_and_stack_images(curr_state_imgs_paths)

            '''
            here, when we run the model on the screenshot images, we get the q-values of action.
            Once check whether we receive a list of list or just a list.
            ex:- [[1, 2, 3, 4]] or [1, 2, 3, 4]
            '''
            curr_state_label = self.checkpoint.model(curr_state_imgs, training = False)[action_index]
            actual_labels.append(curr_state_label)

        return actual_labels


    
    def loss_function(self, target_labels, actual_labels):
        return np.square(np.asarray(target_lables) - np.asarray(actual_labels))
    
    
    def train_step(self):
        target_label_values_for_samples = self.get_target_labels_for_samples(buffer_samples)
        actual_label_values_for_samples = self.get_actual_labels_for_samples(buffer_samples)
        
        if len(target_label_values_for_samples) == len(actual_label_values_for_samples):
            
            with tf.GradientTape(persistent = True) as params_tape:
                loss = self.loss_function(target_label_values_for_samples, actual_label_values_for_samples)
            
            model_gradients = params_tape.gradient(loss, self.checkpoint.model.trainable_variables)
            self.checkpoint.optimizer.apply_gradients(zip(model_gradients, self.checkpoint.model.trainable_variables))
            
            return loss
                
                
            else:
                print('there is a problem with the actual and target variables')
                return -1
    
    

    def train(self):
        '''
        For the first 50000 seteps, just let the agent choose random actions and explore very
        extensively so tha it could have a vast experience buffer
        '''
        if self.num_of_steps_completed_in_obv_period < self.observation_period_threshold:
            self.start_exploration_in_observation_period()

        self.restore_checkpoint()
        epochs_completed = self.checkpoint.curr_epoch.numpy()
        epochs_remaining = config.NUM_EPOCHS - epochs_completed - 1

        for epoch in range(epochs_remaining):
            curr_epoch = self.checkpoint.curr_epoch.numpy()

            self.game_controls    = Game_functions()
            curr_state_imgs_paths = self.game_controls.curr_screenshots_paths
            curr_reward           = self.game_controls.curr_reward
            end_game              = self.game_controls.end_game
#             curr_state           = self.preprocess_and_stack_images(curr_game_imgs_paths)

            '''If the current state is not terminal i.e. not end_game'''
            while not end_game: 
                action_to_be_taken = self.epsilon_greedy_policy_for_action(curr_state_imgs_paths)
                next_state_imgs_paths, end_game, start_training_gap_period = self.add_experiences_to_buffers_after_running_game_for_one_step(action_to_be_taken, curr_state_imgs_paths)
                curr_state_imgs_paths = next_state_imgs_paths

                '''Train the model on random samples from both the experience buffers'''
                self.adjust_buffer_sample_rate(epoch)
                buffer_1_samples, buffer_2_samples = self.sample_mini_batches_from_experience_buffers()
                loss = self.train_step(buffer_1_samples + buffer_2_samples)
                
                
                
                
            
            
    
    
            
            


def train_network():
    game_controls = Game_functions()
    curr_game_imgs_, curr_reward, end_game = game_controls.curr_screenshot_path, game_controls.curr_reward, game_controls.end_game
    
    
    